In [4]:
import pandas as pd
import numpy as np
import os as os
import aux.acessos as ac
import aux.aed as aed
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

## ATIVOS

In [20]:
query_ativo = f"""

with seg_consumo_with_dict as (
	select distinct
			a.mes_ref
			,a.customer_id
			,a.mcc as mcc1
			,a.labels
			,b.*
	from marketplace_sandbox_zone.mktp_model_segmentacao_mcc_analytical_flex_1m as a
	left join marketplace_sandbox_zone.mktp_dict_model_segmentacao_mcc_flex_1m as b
	on
			(a.mcc = b.mcc and a.mes_ref = b.mes_ref and a.labels = b.labels)
	where	a.mes_ref = '2023-01'
	order by 1,2,3
)
, seg_consumo_agg as (
select 
	customer_id
	, MAX(case 
		when mcc1 = 'LOJAS ESPECIALIZADAS' then '5 - LOJAS ESPECIALIZADAS'
		when mcc1 = 'SAUDE' then '4 - SAUDE'
		when mcc1 = 'SERVICOS DE ALIMENTACAO' then '3 - SERVICOS DE ALIMENTACAO'
		when mcc1 = 'LOJAS DE DEPARTAMENTO' then '2 - LOJAS DE DEPARTAMENTO'
		when mcc1 = 'SUPERMERCADO' then '1 - SUPERMERCADO' 
		else '0 - OUTROS' end) as segmento_mcc_agg
from seg_consumo_with_dict
	group by 1
)
, ultimo_ie as (
	select distinct
	cd_cpf, max(cd_yearmonth) over (partition by 1) as ult_yearmonth
	from customer_curated_zone.ca_model_ie_cliente
)
, ultima_compra as (
	select 
	id_customer
	,max(cd_yearmonth) as ult_compra_mes
	from 
	customer_curated_zone.ca_book_cartao
	group by 1
)
,  info_cliente as	(
	 select 
	 id_customer
	 , cpf 
	, to_date(concat(cast(min(cd_yearmonth) as varchar),'01'), 'yyyymmdd') as safra
	from 
	customer_curated_zone.ca_book_cliente 	
	group by 
		1,2)
, base_final as (
select 
*
, case 
	when grupo_2 = 10 then	'Comunicação Genérica'
	when grupo_2 = 0 then   'Sem comunicação'
	else 'Comunicação MCC'
end as grupo_comunicacao
from 
	(
select 
	a.cpf
 	, b.safra
 	, ie.nr_score_ie
  	, ui.ult_yearmonth
	, case 
		when flag_pmm = 'churn' then '3 - Churn'
		else propensao end as propensao
	, grupo
	, case 
		when flag_pmm = 'churn' then 'Inativo'
		when flag_pmm = 'pré-churn' then 'Ativo'
		when a.ds_status_cliente = '0c - Inativo' then 'Inativo'
		when a.ds_status_cliente is not null then a.ds_status_cliente
	else 'Inativo' end as status
	, case 
		when grupo = 'Teste' then NTILE (10) OVER (partition by c.segmento_mcc_agg ORDER BY random())	
		when grupo <> 'Teste' then 0
		end as grupo_2
	, case when c.segmento_mcc_agg is null then '0 - OUTROS' else c.segmento_mcc_agg end as segmento_mcc_agg
	from cxm_sandbox_zone.pre_fatura_turbinada_flag_pmm a
	left join info_cliente b on (b.cpf = a.cpf)
	left join seg_consumo_agg c on (c.customer_id = b.id_customer)
 	left join ultimo_ie ui on (b.cpf = ui.cd_cpf)
 	left join customer_curated_zone.ca_model_ie_cliente ie on (b.cpf = ie.cd_cpf and ie.cd_yearmonth = ui.ult_yearmonth)
	)
)
select
cpf
, max(grupo_comunicacao) as grupo_comunicacao
, max(segmento_mcc_agg) as segmento_mcc_agg
, max(nr_score_ie) as nr_score_ie
, max(ult_yearmonth) as yearmonth_ie
, max(status) as status
, max(safra) as safra
from base_final
where status = 'Ativo'
"""

In [21]:
grupos = [ '1 - SUPERMERCADO', '2 - LOJAS DE DEPARTAMENTO', '3 - SERVICOS DE ALIMENTACAO', '4 - SAUDE', '5 - LOJAS ESPECIALIZADAS', '0 - OUTROS']

ativos = pd.DataFrame()

for i in grupos:
     query_ativo_new = query_ativo + "and segmento_mcc_agg = '" 
     query_ativo_new = query_ativo_new + i + "'"
     query_ativo_new = query_ativo_new + "group by 1"
     print(i)
     temp_df = ac.df_athena_q('flavia-costa', query_ativo_new)
     ativos = ativos.append(temp_df) 

1 - SUPERMERCADO


/tmp/ipykernel_1038431/2364746422.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ativos = ativos.append(temp_df)


2 - LOJAS DE DEPARTAMENTO


/tmp/ipykernel_1038431/2364746422.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ativos = ativos.append(temp_df)


3 - SERVICOS DE ALIMENTACAO


...

/tmp/ipykernel_1038431/2364746422.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ativos = ativos.append(temp_df)


5 - LOJAS ESPECIALIZADAS


/tmp/ipykernel_1038431/2364746422.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ativos = ativos.append(temp_df)


0 - OUTROS


/tmp/ipykernel_1038431/2364746422.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ativos = ativos.append(temp_df)


In [22]:
len(ativos)

1728480

In [23]:
ativos.value_counts('segmento_mcc_agg')

segmento_mcc_agg
0 - OUTROS                     774858
4 - SAUDE                      264899
3 - SERVICOS DE ALIMENTACAO    222620
2 - LOJAS DE DEPARTAMENTO      217268
1 - SUPERMERCADO               124666
5 - LOJAS ESPECIALIZADAS       124169
dtype: int64

In [24]:
ativos.value_counts('status')

status
Ativo    1728480
dtype: int64

In [25]:
pd.crosstab(ativos.segmento_mcc_agg, ativos.grupo_comunicacao)

grupo_comunicacao,Comunicação Genérica,Comunicação MCC,Sem comunicação
segmento_mcc_agg,,,
0 - OUTROS,64699,632263,77896
1 - SUPERMERCADO,11213,100946,12507
2 - LOJAS DE DEPARTAMENTO,19573,176116,21579
3 - SERVICOS DE ALIMENTACAO,20082,180201,22337
4 - SAUDE,23773,214861,26265
5 - LOJAS ESPECIALIZADAS,11235,100439,12495


In [26]:
ativos.head(3)

,cpf,grupo_comunicacao,segmento_mcc_agg,nr_score_ie,yearmonth_ie,status,safra
0,91908973234,Comunicação MCC,1 - SUPERMERCADO,29.713168,202302.0,Ativo,2021-09-01
1,14258158470,Comunicação MCC,1 - SUPERMERCADO,50.899609,202302.0,Ativo,2021-10-01
2,22377212468,Comunicação MCC,1 - SUPERMERCADO,59.104737,202302.0,Ativo,2022-02-01


In [27]:
ativos.to_pickle('ativos_fatura_premiada.pkl')

In [30]:
for i in grupos:
    ativos[ativos['segmento_mcc_agg'] == i] \
    .query('grupo_comunicacao == "Comunicação MCC"') \
    [['cpf', 'segmento_mcc_agg', 'status']].to_csv(f'Ativos_fatura_premiada_{i}.csv')

In [32]:
ativos \
.query('grupo_comunicacao == "Comunicação Genérica"') \
.to_csv(f'Ativos_fatura_premiada_Comunicação_Genérica.csv')

## INATIVOS

In [33]:
query_inativo = f"""

with seg_consumo_with_dict as (
	select distinct
			a.mes_ref
			,a.customer_id
			,a.mcc as mcc1
			,a.labels
			,b.*
	from marketplace_sandbox_zone.mktp_model_segmentacao_mcc_analytical_flex_1m as a
	left join marketplace_sandbox_zone.mktp_dict_model_segmentacao_mcc_flex_1m as b
	on
			(a.mcc = b.mcc and a.mes_ref = b.mes_ref and a.labels = b.labels)
	where	a.mes_ref = '2023-01'
	order by 1,2,3
)
, seg_consumo_agg as (
select 
	customer_id
	, MAX(case 
		when mcc1 = 'LOJAS ESPECIALIZADAS' then '5 - LOJAS ESPECIALIZADAS'
		when mcc1 = 'SAUDE' then '4 - SAUDE'
		when mcc1 = 'SERVICOS DE ALIMENTACAO' then '3 - SERVICOS DE ALIMENTACAO'
		when mcc1 = 'LOJAS DE DEPARTAMENTO' then '2 - LOJAS DE DEPARTAMENTO'
		when mcc1 = 'SUPERMERCADO' then '1 - SUPERMERCADO' 
		else '0 - OUTROS' end) as segmento_mcc_agg
from seg_consumo_with_dict
	group by 1
)
, ultimo_ie as (
	select distinct
	cd_cpf, max(cd_yearmonth) over (partition by 1) as ult_yearmonth
	from customer_curated_zone.ca_model_ie_cliente
)
, ultima_compra as (
	select 
	id_customer
	,max(cd_yearmonth) as ult_compra_mes
	from 
	customer_curated_zone.ca_book_cartao
	group by 1
)
,  info_cliente as	(
	 select 
	 id_customer
	 , cpf 
	, to_date(concat(cast(min(cd_yearmonth) as varchar),'01'), 'yyyymmdd') as safra
	from 
	customer_curated_zone.ca_book_cliente 	
	group by 
		1,2)
, base_final as (
select 
*
, case 
	when grupo_2 = 10 then	'Comunicação Genérica'
	when grupo_2 = 0 then   'Sem comunicação'
	else 'Comunicação MCC'
end as grupo_comunicacao
from 
	(
select 
	a.cpf
 	, b.safra
 	, ie.nr_score_ie
  	, ui.ult_yearmonth
	, case 
		when flag_pmm = 'churn' then '3 - Churn'
		else propensao end as propensao
	, grupo
	, case 
		when flag_pmm = 'churn' then 'Inativo'
		when flag_pmm = 'pré-churn' then 'Ativo'
		when a.ds_status_cliente = '0c - Inativo' then 'Inativo'
		when a.ds_status_cliente is not null then a.ds_status_cliente
	else 'Inativo' end as status
	, case 
		when grupo = 'Teste' then NTILE (10) OVER (partition by c.segmento_mcc_agg ORDER BY random())	
		when grupo <> 'Teste' then 0
		end as grupo_2
	, case when c.segmento_mcc_agg is null then '0 - OUTROS' else c.segmento_mcc_agg end as segmento_mcc_agg
	from cxm_sandbox_zone.pre_fatura_turbinada_flag_pmm a
	left join info_cliente b on (b.cpf = a.cpf)
	left join seg_consumo_agg c on (c.customer_id = b.id_customer)
 	left join ultimo_ie ui on (b.cpf = ui.cd_cpf)
 	left join customer_curated_zone.ca_model_ie_cliente ie on (b.cpf = ie.cd_cpf and ie.cd_yearmonth = ui.ult_yearmonth)
	)
)
select
cpf
, max(grupo_comunicacao) as grupo_comunicacao
, max(segmento_mcc_agg) as segmento_mcc_agg
, max(nr_score_ie) as nr_score_ie
, max(ult_yearmonth) as yearmonth_ie
, max(status) as status
, max(safra) as safra
from base_final
where status = 'Inativo'
"""

In [35]:
grupos = ['Comunicação Genérica','Comunicação MCC','Sem comunicação']

inativos = pd.DataFrame()
for i in grupos:
     query_inativo_new = query_inativo + "and grupo_comunicacao = '" 
     query_inativo_new = query_inativo_new + i + "'"
     query_inativo_new = query_inativo_new + "group by 1"
     print(i)
     temp_df = ac.df_athena_q('flavia-costa', query_inativo_new)
     inativos = inativos.append(temp_df) 

Comunicação Genérica


/tmp/ipykernel_1038431/4098941072.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inativos = inativos.append(temp_df)


Comunicação MCC


/tmp/ipykernel_1038431/4098941072.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inativos = inativos.append(temp_df)


Sem comunicação


/tmp/ipykernel_1038431/4098941072.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inativos = inativos.append(temp_df)


In [36]:
inativos.value_counts('status')

status
Inativo    916338
dtype: int64

In [37]:
inativos.value_counts('grupo_comunicacao')

grupo_comunicacao
Comunicação MCC         732744
Comunicação Genérica     95054
Sem comunicação          88540
dtype: int64

In [38]:
len(inativos)

916338

In [39]:
inativos.head(3)

,cpf,grupo_comunicacao,segmento_mcc_agg,nr_score_ie,yearmonth_ie,status,safra
0,13364903735,Comunicação Genérica,1 - SUPERMERCADO,NaN,202302.0,Inativo,2021-03-01
1,60598266364,Comunicação Genérica,1 - SUPERMERCADO,NaN,202302.0,Inativo,2021-03-01
2,86127770548,Comunicação Genérica,1 - SUPERMERCADO,8.426771,202302.0,Inativo,2019-04-01


In [40]:
inativos.to_pickle('inativos_fatura_premiada.pkl')

In [41]:
len(inativos.query('grupo_comunicacao != "Sem comunicação"'))

827798

In [43]:
inativos \
.query('grupo_comunicacao != "Sem comunicação"')[['cpf', 'segmento_mcc_agg', 'status']] \
.to_csv(f'Inativos_fatura_premiada.csv')

In [ ]:
#infos = pd.read_pickle('stonks_picke.pkl')